In [51]:
import numpy as np
import pandas as pd
import matplotlib.dates as md
import matplotlib.pyplot as plt
import seaborn as sns
import io

In [52]:
from math import sin
from math import pi
from numpy import arange
from numpy import vstack
from numpy import argmax
from numpy import asarray
from numpy.random import normal
from numpy.random import random
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from warnings import catch_warnings
from warnings import simplefilter
from matplotlib import pyplot

In [53]:
# objective function
def objective(x, noise=0.1):
	noise = normal(loc=0, scale=noise)
	return (x**2 * sin(5 * pi * x)**6.0) + noise

In [54]:
# surrogate or approximation for the objective function
def surrogate(model, X):
	# catch any warning generated when making a prediction
	with catch_warnings():
		# ignore generated warnings
		simplefilter("ignore")
		return model.predict(X, return_std=True)

In [55]:
# probability of improvement acquisition function
def acquisition(X, Xsamples, model):
	# calculate the best surrogate score found so far
	yhat, _ = surrogate(model, X)
	best = max(yhat)
	# calculate mean and stdev via surrogate function
	mu, std = surrogate(model, Xsamples)
	mu = mu[:, 0]
	# calculate the probability of improvement
	probs = norm.cdf((mu - best) / (std+1E-9))
	return probs

In [56]:
# optimize the acquisition function
def opt_acquisition(X, y, model):
	# random search, generate random samples
	Xsamples = random(100)
	Xsamples = Xsamples.reshape(len(Xsamples), 1)
	# calculate the acquisition function for each sample
	scores = acquisition(X, Xsamples, model)
	# locate the index of the largest scores
	ix = argmax(scores)
	return Xsamples[ix, 0]

In [57]:
# plot real observations vs surrogate function
def plot(X, y, model):
	# scatter plot of inputs and real objective function
	pyplot.scatter(X, y)
	# line plot of surrogate function across domain
	Xsamples = asarray(arange(0, 1, 0.001))
	Xsamples = Xsamples.reshape(len(Xsamples), 1)
	ysamples, _ = surrogate(model, Xsamples)
	pyplot.plot(Xsamples, ysamples)
	# show the plot
	pyplot.show()

In [85]:
df = pd.read_csv (r'D:\cerealsugar.csv')
print (df)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             x
36.3 33.2 39 37.3 40.7 38.4 35.8 36 37.9 42.6 40 43.5 36.6 33.7 40.5 38.7 36.2 41.6 37.8 40.5 38.9 36 41.1 40 35.5 34.3 33 36.9 36.3 38.5 35.1 38.7 34 42.3 41.4 38.2 38.3 37.4 37.5 36.5 34.8 38.1 41 35.4 35 37.9 39.1 33.3 40.4 34.9 33.5 35.8 36.6 38.3 33.1 37.4 39.9 38.4 36.1 33.6 35.1 37.4 38.2 39 31.6 36.1 36.6 36.4 39.9 38.8 39.1 34.7 37.3 39.1 38.8 38.7 36.5 37.3 39.9 34 39.4 36.4 34.8 41.1 33.8 41.4 39.1 38.3 37.7 38.4 40.9 36.5 40.5 39.7 40.6 38.9 39.3 37.1 35.6  36.8


In [86]:
# sample the domain sparsely with noise
X = x in df
y = asarray([objective(x) for x in X])
# reshape into rows and cols
X = X.reshape(len(X), 1)
y = y.reshape(len(y), 1)
# define the model
model = GaussianProcessRegressor()
# fit the model
model.fit(X, y)
# plot before hand
plot(X, y, model)
# perform the optimization process
for i in range(100):
	# select the next point to sample
	x = opt_acquisition(X, y, model)
	# sample the point
	actual = objective(x)
	# summarize the finding
	est, _ = surrogate(model, [[x]])
	print('>x=%.2f, f()=%.2f, actual=%.2f' % (x, est, actual))
	# add the data to the dataset
	X = vstack((X, [[x]]))
	y = vstack((y, [[actual]]))
	# update the model
	model.fit(X, y)

# plot all samples and the final surrogate function
plot(X, y, model)
# best result
ix = argmax(y)
print('Best Result: x=%.2f, y=%.2f' % (X[ix], y[ix]))

TypeError: 'bool' object is not iterable

[0.67041755 0.88545996 0.37079221 0.26073466 0.56271214 0.16961589
 0.49648228 0.70492397 0.57505945 0.17265118 0.08661565 0.79527211
 0.89414038 0.35760376 0.05652033 0.75715817 0.54733343 0.88768044
 0.14435601 0.58043941 0.20599223 0.50312452 0.9305545  0.19371822
 0.86308745 0.86242223 0.8785647  0.56612407 0.29684204 0.62436992
 0.51812858 0.76793831 0.94230387 0.49828729 0.64934695 0.2388211
 0.04295765 0.06613283 0.63616589 0.98024584 0.65931349 0.05625138
 0.24487392 0.74898809 0.82097041 0.90319497 0.73228718 0.57851834
 0.62790669 0.97316403 0.27971574 0.36287665 0.97039942 0.49313654
 0.05547191 0.08200943 0.96846723 0.08629515 0.40271642 0.19871983
 0.11389172 0.77460808 0.86995246 0.40847147 0.01653969 0.89032007
 0.13876164 0.46229745 0.04698015 0.86777507 0.88494164 0.41674805
 0.9169173  0.60397842 0.75329509 0.2001011  0.48427018 0.9621149
 0.15897601 0.67008141 0.19759842 0.04163055 0.05115518 0.38382815
 0.52178732 0.86306246 0.93838184 0.81826566 0.74766948 0.221002